In [2]:
import cv2
import time
import numpy as np
import HandTrackingModule as htm    # 03. 導入自訂的HandTrackingModule.py
# 07. 匯入sleep
from time import sleep
import random
# 09. 匯入鍵盤控制
from pynput.keyboard import Controller
import threading
# 01. 取得鏡頭影像的基本架構
wCam, hCam = 1280, 720
cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)

# 02. FPS時間設定
pTime = 0

def job():
    a = random.randint(1,99)
    print(a)
    b = int(input('輸入 1～99 的數字：'))
    while True:
    
        if b < a:
            b = int(input('數字太小囉！再試一次吧：'))
        elif b > a:
            b = int(input('數字太大囉！再試一次吧：'))
        else:
            print('答對囉！')
            break;
    time.sleep(1)

# 建立一個子執行緒
t = threading.Thread(target = job)
# 執行該子執行緒
t.start()
# 03. 使用 HandTrackingModule.py函式擷取手部資訊
detector = htm.handDetector(detectionCon=0.75, maxHands=1)
# 08. 設定顯示透明度
def drawAll(img, buttonList):
    imgNew = np.zeros_like(img, np.uint8)
    for button in buttonList:
        button.draw(imgNew)
    
    out = img.copy()
    alpha =0.2
    mask = imgNew.astype(bool)
    out[mask] =  cv2.addWeighted(img, alpha, imgNew, 1-alpha, 0)[mask]
    return out

# 04. 在畫面中建立一組虛擬鍵盤按鈕
class Button():
    def __init__(self, pos, text, size=[85,85]):
        self.pos = pos
        self.size = size
        self.text = text
    
    def draw(self, img,ccolors=(255,0,255), tcolors=(0,255,255)):
        #print(self.text)
        if self.text == "black" or self.text == "enter":
            x, y = self.pos
            w, h = self.size
            cv2.rectangle(img, self.pos, (x+w, y+h), tcolors, thickness=10)
            cv2.rectangle(img, self.pos, (x+w, y+h), ccolors, cv2.FILLED)
            
        else:
            x, y = self.pos
            w, h = self.size
            cv2.rectangle(img, self.pos, (x+w, y+h), tcolors, thickness=10)
            cv2.rectangle(img, self.pos, (x+w, y+h), ccolors, cv2.FILLED)
            cv2.putText(img, self.text, (self.pos[0]+18, self.pos[1]+68), cv2.FONT_HERSHEY_PLAIN, 5, (255, 255, 255), 5)
        #cv2.line(img, 開始座標, 結束座標, 顏色, 線條寬度)
        return img
# 04.宣告一組虛擬鍵盤按鈕
keys=[("0", "1", "2", "3", "4", "5"),
      ("6", "7", "8", "9","10")]
buttonList = []
for n in range(0, 2):
    for x, key in enumerate(keys[n]):
        buttonList.append(Button([100*x+50, 100*n+100], key))
# 07. 宣告輸入字串
finalText = ""
# 09. 鍵盤控制
keyboard = Controller()

while True:
    success, img = cap.read()
    # 水平翻轉畫面
    img = cv2.flip(img, 1) 
    # 03. 偵測手並傳回座標位置
    img = detector.findHands(img, draw = False)
    # 04. 在畫面中建立一組虛擬鍵盤按鈕
    #for button in buttonList:
        #img = button.draw(img)
    img = drawAll(img, buttonList)  # 08. 設定透明鍵盤

    lmList, bbox = detector.findPosition(img)
    # 05. 檢查手指是否在某一個按鈕位置
    if lmList:
        for button in buttonList:
            x, y = button.pos
            w, h = button.size
            if (x < lmList[8][1] < x+w) and (y < lmList[8][2] < y+h):
                img = button.draw(img, ccolors=(0, 255, 0))
                # 06. 點擊模式:食指與中指併攏
                length, _, _ = detector.findDistance(8, 12, img)
                cv2.putText(img, f'L:{int(length)}', (180, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 3)
                #print(length)
                # 07. 兩指距離短則輸入字元
                if length < 50:
                    keyboard.press(button.text)
                    img = button.draw(img, ccolors=(0, 255, 0)) 
                    finalText += button.text 
                    sleep(0.15)
                    # 09. 鍵盤控制
    # 07. 顯示文字
    cv2.rectangle(img, (50, 550), (1000, 650), (175, 255, 175), cv2.FILLED)
    cv2.putText(img, finalText, (60, 600), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,0), 3)
    # 02. 顯示FPS (frame rate)
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img, f'FPS:{int(fps)}', (wCam-120, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 3)
    cv2.imshow("Image", img)
    # 01. 按Q離
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()


65


In [ ]:
import cv2

# 讀取圖像
image = cv2.imread('koi.jpg')

# 將圖像轉換為灰度
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 應用高斯模糊以減少噪聲
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# 使用Canny邊緣檢測算法檢測邊緣
edges = cv2.Canny(blurred, 50, 150)

# 找到圖像中的輪廓
contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 繪製輪廓
cv2.drawContours(image, contours, -1, (0, 255, 0), 2)

# 顯示原始圖像和檢測結果
cv2.imshow('Original Image', image)
cv2.imshow('Edges', edges)
cv2.waitKey(0)
cv2.destroyAllWindows()
